# Sudoku Solver V0.1

This is my implemetation* of Peter Norvig's "[Solving Every Sudoku Puzzle](http://www.norvig.com/sudoku.html)".

*And by implementation, I mean I copy and pasted his code and stepped through it line by line till I had some semblance of an understanding of the algorithm.

### Motivation

60% learning exercise, 40% taking revenge on years of failed and partially completed 4+ star Sudoku puzzles. Let's get to it!

In [24]:
import pandas as pd
import plotly as py
import plotly.graph_objs as go

py.offline.init_notebook_mode(connected=True)

#Defining global variables used to index puzzle
digs   = '123456789'             #possible digits in a 9x9 sudoku puzzle
rows     = 'ABCDEFGHI'           #alphabet labels for each of 9 rows
cols     = digs                  #numeric labels for each of 9 columns


sqrs = [r+c for r in rows for c in cols]
row_units = [[r + c for c in cols]for r in rows]
col_units = [[r + c for r in rows]for c in cols]

rsubs = ['ABC','DEF','GHI']
csubs = ['123','456','789']

box_units = []
for rsub in rsubs:
    for csub in csubs:
        box_units.append([r+c for r in rsub for c in csub])
        

unitlist = (row_units+col_units+box_units)

unitdict = {}
for sq in sqrs:
    for u in unitlist:
        if sq in u:
            if sq not in unitdict:
                unitdict[sq]=[]
            unitdict[sq].append(u)

peers = {}
for s in sqrs:
    unitset = set()
    for unit in unitdict[s]:
        for square in unit:
            if square != s:
                unitset.add(square)
    peers[s] = unitset


### The 'Easy' Part

#### Read in and validate the puzzle

Okay, now let's read in the puzzle. I'm going to go with the simplest solution and take a list of 81 characters (81 squares in a 9x9 puzzle) as an input; entered row-wise, using periods or dashes to represent unknown (blank) cells.


In [25]:
#Here are some examples to play with (I'll save you the typing)
examples = ['4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......', #easy peasy
            '53..7....6..195....98....6.8...6...34..8.3..17...2...6.6....28....419..5....8..79', #easy peasy
            '.....6....59.....82....8....45........3........6..3.54...325..6..................'] #super hard!!! will take a while to comput

raw_puzzle = examples[0]

#input("Enter the Sudoku Puzzle you'd like solved as a string of 81 characters\n(Use periods (.) or dashes (-) to denote unknown values):")

puzzle = [c for c in raw_puzzle if c in digs or c in '.-']

assert len(puzzle) == 81, "Your entry is not 81 characters long! Please try again!"

puzzledict = dict(zip(sqrs,puzzle))

Now that we have a Sudoku puzzle to solve, let's figure out a way to visualize it.

Peter's highly functional (and super "[Pythonic](https://stackoverflow.com/questions/25011078/what-does-pythonic-mean)") code utilizes a simple 'display' function as defined below to print the puzzle as plain text:


In [29]:
def display(p):
    width = 1+max(len(p[s]) for s in sqrs)
    divider = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(p[r+c].center(width)+('|' if c in '36' else '') for c in cols))
        if r in 'CF': 
            print(divider)
    return

Here comes my one and only new contribution to Mr.Norvig's approach, and it's purely aesthetic. 

In the spirit of learning, exploring yet another Python library, and of course, prettification, here is my entirely un-Pythonic and verbose display function using [Plotly](https://plot.ly/d3-js-for-python-and-pandas-charts/)'s Python library:

In [ ]:
def display_as_table(p):
    df = pd.DataFrame(columns=list(cols), index = list(rows))
    
    for r in rows:
        for c in cols:
            df.loc[r,c] = p[r+c]
    
    vals = [df[c] for c in list(cols)]
    vals.insert(0,pd.Series(list(rows)))
    h = 30
    
    indexcolor = '#ffffff'
    linecolor = '#ffffff'
    darkcellcolor = '#ffe0b3'
    lightcellcolor = '#fff5e6'
    dld = [darkcellcolor]*3+[lightcellcolor]*3+[darkcellcolor]*3
    ldl = [lightcellcolor]*3+[darkcellcolor]*3+[lightcellcolor]*3
    
    cellcolors = ([indexcolor]*9,dld,dld,dld,ldl,ldl,ldl,dld,dld,dld)
    
    trace = go.Table(
        columnwidth = h,
        header=dict(values = ['']+(list(cols)),
                    fill = dict(color=indexcolor),
                    line = dict(color=linecolor),
                    align = ['center'] * (len(list(cols))+1),
                    height = h),
        cells=dict(values = vals,
                   fill = dict(color=cellcolors),
                   line = dict(color=linecolor),
                   align = ['center']*(len(list(rows))+1),
                   height = h))

    layout = dict(autosize=True,
                  width=480, 
                  height=500, 
                  margin=go.Margin(l=50,
                                    r=50,
                                    b=50,
                                    t=50,
                                    pad=4
                 ))
    
    data = [trace]
    fig= dict(data=data, layout=layout)
    
    py.offline.iplot(fig)

    return
    

### The 'Brain-hurty' Part



In [27]:
def possible_values(puzzle):
    values = dict((s, digs) for s in sqrs)
    for s,d in puzzledict.items():
        if d in digs and not assign(values, s, d):
            return False
    
    display_as_table(values)
    return values

def eliminate(values, s, d):

    if d not in values[s]:
        return values 
    values[s] = values[s].replace(d,'')
    
    if len(values[s]) == 0:
        return False
    elif len(values[s]) == 1:
        d2 = values[s]
        if not all(eliminate(values, s2, d2) for s2 in peers[s]):
            return False
    
    for u in unitdict[s]:
        dplaces = [s for s in u if d in values[s]]
        if len(dplaces) == 0:
            return False
        elif len(dplaces) == 1:
            if not assign(values, dplaces[0], d):
                return False
    return values

def assign(values, s, d):
    trial_values = values[s].replace(d, '')
    if all(eliminate(values, s, d2) for d2 in trial_values):
        return values
    else:
        return False

def search(values):
    if values is False:
        return False 
    if all(len(values[s]) == 1 for s in sqrs): 
        return values 
    
    n,s = min((len(values[s]), s) for s in sqrs if len(values[s]) > 1)
    return some(search(assign(values.copy(), s, d)) for d in values[s])

def some(seq):
    for e in seq:
        if e: return e
    return False

def solve(puzzle): 
    
    return display_as_table(search(possible_values(puzzle)))



In [28]:
solve(puzzle)